# lolapi.py Usage

Below is a demonstration of how to use the lolapi.py wrapper to retreive match data from the Riot Games API.

In [1]:
import lolapi as la

Input `apikey` and `summonername`.

API key is assigned through the [Riot Developer Portal](https://developer.riotgames.com/).

Summoner name is the player's name as it appears in game (capitalization must match).

In [2]:
apikey = ''
summonername = ''

Encrypted account ID is retrieved using the `retrieve_accountid()` function:

In [3]:
accountid = la.retrieve_accountid(apikey, summonername)

Pass the `accountid` to collect matches for that account.

`queue` parameter is optional. In this example 450 is used to filter for the 'ARAM' game mode. To find queue values for other game modes please refer to the Riot API documentation.

If collecting over 1000 matches the request may receive a timeout error. Riot API only allows for 100 requests per 2 minutes.

In [4]:
account_matches_df = la.retrieve_account_matches(apikey, accountid, queue=450)
print(str(len(account_matches_df)) + ' matches retrieved.')

600 matches retrieved.


In [5]:
for column in account_matches_df.columns:
    print(column)

champion
gameId
lane
platformId
queue
role
season
timestamp


Use the 'gameId' column from the retrieved matches as an input for the `retrieve_match_data()` function. This will bring in the detailed data for each match. Due to the request volume restriction the program will pause for 2 mins when it receives a timeout or resource exceeded response. If it receives a second error after waiting it will cancel.

Use the `limit` parameter to select only a specific number of matches. These are collected in the same order as the list of match IDs given. If given limit is over the total possible matches it will collect all matches.

In [6]:
match_detail_data = la.retrieve_match_data(apikey, account_matches_df['gameId'], limit=5)

5 matches collected.


The collected match data can be parsed using the `parse_match_data()` function. Note, this only collects overall information such as gameID, duration, and player specific information and statistics for the given summoner name. Other overall statistics and info for other players are not collected in this function.

Optional: Pass the account matches dataframe in the account_matches_df argument to merge the match detail data with the summary data collected from `retrieve_account_matches()`.

In [7]:
match_data_df = la.parse_match_data(summonername, match_detail_data, account_matches_df)
match_data_df.head()

,assists,champLevel,combatPlayerScore,damageDealtToObjectives,damageDealtToTurrets,damageSelfMitigated,deaths,doubleKills,duration,firstBloodAssist,...,visionScore,visionWardsBoughtInGame,win,champion,lane,platformId,queue,role,season,timestamp
0,39,18,0,1627,1627,64909,9,0,1126,False,...,0,0,True,33,NONE,NA1,450,DUO_SUPPORT,13,1597425688147
1,19,15,0,1519,1519,3668,5,0,1039,False,...,0,0,True,101,NONE,NA1,450,DUO_SUPPORT,13,1597424507444
2,30,16,0,2433,2433,25581,6,0,974,False,...,0,0,True,3,NONE,NA1,450,DUO_SUPPORT,13,1597382554359
3,34,18,0,0,0,39709,7,0,1209,False,...,0,0,False,201,MID,NA1,450,DUO_SUPPORT,13,1597381186352
4,13,16,0,1241,1241,8365,9,1,1054,False,...,0,0,False,110,NONE,NA1,450,DUO_SUPPORT,13,1597379976671


In [8]:
for column in match_data_df.columns:
    print(column)

assists
champLevel
combatPlayerScore
damageDealtToObjectives
damageDealtToTurrets
damageSelfMitigated
deaths
doubleKills
duration
firstBloodAssist
firstBloodKill
firstInhibitorAssist
firstInhibitorKill
firstTowerAssist
firstTowerKill
gameId
goldEarned
goldSpent
inhibitorKills
item0
item1
item2
item3
item4
item5
item6
killingSprees
kills
largestCriticalStrike
largestKillingSpree
largestMultiKill
longestTimeSpentLiving
magicDamageDealt
magicDamageDealtToChampions
magicalDamageTaken
neutralMinionsKilled
objectivePlayerScore
participantId
pentaKills
perk0
perk0Var1
perk0Var2
perk0Var3
perk1
perk1Var1
perk1Var2
perk1Var3
perk2
perk2Var1
perk2Var2
perk2Var3
perk3
perk3Var1
perk3Var2
perk3Var3
perk4
perk4Var1
perk4Var2
perk4Var3
perk5
perk5Var1
perk5Var2
perk5Var3
perkPrimaryStyle
perkSubStyle
physicalDamageDealt
physicalDamageDealtToChampions
physicalDamageTaken
playerScore0
playerScore1
playerScore2
playerScore3
playerScore4
playerScore5
playerScore6
playerScore7
playerScore8
playerScore9
q

Champion data is another valuable piece of information and can be retrieved using the `retrieve_champ_info()` function. This collects overall data for the champion not related to specific match performance.

In [9]:
champ_df = la.retrieve_champ_info()
champ_df.head()

,attack,defense,difficulty,id,magic,name,role
0,8,4,4,266,3,Aatrox,Fighter
1,3,4,5,103,8,Ahri,Mage
2,5,3,7,84,8,Akali,Assassin
3,6,9,7,12,5,Alistar,Tank
4,2,6,3,32,8,Amumu,Tank


Use pandas to merge the champion data to the match detail data, providing additional champion specific information:

In [10]:
import pandas as pd
full_match_data = pd.merge(match_data_df, champ_df, left_on='champion', right_on='id')
full_match_data.head()

,assists,champLevel,combatPlayerScore,damageDealtToObjectives,damageDealtToTurrets,damageSelfMitigated,deaths,doubleKills,duration,firstBloodAssist,...,role_x,season,timestamp,attack,defense,difficulty,id,magic,name,role_y
0,39,18,0,1627,1627,64909,9,0,1126,False,...,DUO_SUPPORT,13,1597425688147,4,10,5,33,5,Rammus,Tank
1,19,15,0,1519,1519,3668,5,0,1039,False,...,DUO_SUPPORT,13,1597424507444,1,3,8,101,10,Xerath,Mage
2,30,16,0,2433,2433,25581,6,0,974,False,...,DUO_SUPPORT,13,1597382554359,1,10,5,3,6,Galio,Tank
3,34,18,0,0,0,39709,7,0,1209,False,...,DUO_SUPPORT,13,1597381186352,3,9,3,201,4,Braum,Support
4,13,16,0,1241,1241,8365,9,1,1054,False,...,DUO_SUPPORT,13,1597379976671,7,3,2,110,4,Varus,Marksman


Export as desired and your detail match data is ready for analysis:

In [11]:
full_match_data.to_csv('full_match_data.csv')